## MNIST 손글씨 분류
### 1. Flatten
: Conv2D를 사용하기 위해 reshape 해줌

In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
# 데이터 준비
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)   # 정답 data: 범주형 0~9

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [3]:
# 이미지(2차원 요소) -> 1차원
# x_train = x_train.reshape(60000,28*28)
# x_test = x_test.reshape(10000,28*28)
y_train = pd.get_dummies(y_train)   # 원핫 인코딩
y_test = pd.get_dummies(y_test)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000, 10)
(10000, 28, 28) (10000, 10)


In [ ]:
# 모델 준비
X = tf.keras.Input(shape=[28,28])   # 768
H = tf.keras.layers.Flatten()(X)
H = tf.keras.layers.Dense(84, activation='swish') (H)
Y = tf.keras.layers.Dense(10, activation='softmax')(H)

model = tf.keras.Model(X,Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.summary()


In [33]:
# 모델 학습
# validation_split으로 학습할 때 test용으로 사용. 아는 문제로 학습하면 accuracy가 높게 나옴.
# batch_size=128 : 6만개의 데이터를 600개로 나눈다. test용 20%를 빼면, 480번 weight를 조정할 기회를 얻음(효율적)
model.fit(x_train, y_train, epochs=10, validation_split=0.1, batch_size=128)

# 학습 결과 : loss: 0.1111 - accuracy: 0.9773 - val_loss: 0.3453 - val_accuracy: 0.9650


Epoch 1/10
422/422 [==============================] - 1s 3ms/step - loss: 0.1889 - accuracy: 0.9634 - val_loss: 0.2997 - val_accuracy: 0.9575
Epoch 2/10
422/422 [==============================] - 1s 3ms/step - loss: 0.1676 - accuracy: 0.9662 - val_loss: 0.2765 - val_accuracy: 0.9610
Epoch 3/10
422/422 [==============================] - 1s 3ms/step - loss: 0.1595 - accuracy: 0.9681 - val_loss: 0.3051 - val_accuracy: 0.9582
Epoch 4/10
422/422 [==============================] - 1s 3ms/step - loss: 0.1575 - accuracy: 0.9695 - val_loss: 0.3273 - val_accuracy: 0.9552
Epoch 5/10
422/422 [==============================] - 1s 3ms/step - loss: 0.1408 - accuracy: 0.9712 - val_loss: 0.3649 - val_accuracy: 0.9607
Epoch 6/10
422/422 [==============================] - 1s 3ms/step - loss: 0.1365 - accuracy: 0.9720 - val_loss: 0.3464 - val_accuracy: 0.9577
Epoch 7/10
422/422 [==============================] - 1s 3ms/step - loss: 0.1316 - accuracy: 0.9732 - val_loss: 0.3029 - val_accuracy: 0.9608
Epoch 

In [ ]:
import matplotlib.pyplot as plt

num = 1244
print(y_test[num:num+1])
print(model.predict(x_test[num:num+1]))

plt.imshow(x_test[num].reshape(28, 28), cmap="gray")
plt.show()


### 2. Convolution
: 특정한 패턴의 특징이 어디서 나타나는지를 확인하는 도구

In [5]:
# Convloution을 하기 위해서는 3채널로 입력
X = tf.keras.Input(shape=[28,28,1])
H = tf.keras.layers.Conv2D(filters=3, kernel_size=5, activation='swish')(X) #(3,24,24,1)
H = tf.keras.layers.Conv2D(6, 5, activation='swish')(H)
H = tf.keras.layers.Flatten()(H)
H = tf.keras.layers.Dense(84, activation='swish')(H)
Y = tf.keras.layers.Dense(10, activation='softmax')(H)

model = tf.keras.Model(X,Y)
model.compile(loss="categorical_crossentropy", metrics='accuracy')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 24, 24, 3)         78        
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 6)         456       
                                                                 
 flatten (Flatten)           (None, 2400)              0         
                                                                 
 dense (Dense)               (None, 84)                201684    
                                                                 
 dense_1 (Dense)             (None, 10)                850       
                                                                 
Total params: 203,068
Trainable params: 203,068
Non-trainable

In [6]:
# 모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.1)

# 학습 결과 - loss: 0.0083 - accuracy: 0.9976 - val_loss: 0.1650 - val_accuracy: 0.9807

Epoch 1/10
422/422 [==============================] - 21s 48ms/step - loss: 1.3559 - accuracy: 0.8881 - val_loss: 0.1207 - val_accuracy: 0.9678
Epoch 2/10
422/422 [==============================] - 17s 40ms/step - loss: 0.0884 - accuracy: 0.9748 - val_loss: 0.0853 - val_accuracy: 0.9757
Epoch 3/10
422/422 [==============================] - 19s 45ms/step - loss: 0.0454 - accuracy: 0.9867 - val_loss: 0.0990 - val_accuracy: 0.9782
Epoch 4/10
422/422 [==============================] - 21s 50ms/step - loss: 0.0262 - accuracy: 0.9922 - val_loss: 0.0900 - val_accuracy: 0.9783
Epoch 5/10
422/422 [==============================] - 19s 46ms/step - loss: 0.0151 - accuracy: 0.9951 - val_loss: 0.1124 - val_accuracy: 0.9800
Epoch 6/10
422/422 [==============================] - 20s 46ms/step - loss: 0.0133 - accuracy: 0.9958 - val_loss: 0.1106 - val_accuracy: 0.9820
Epoch 7/10
422/422 [==============================] - 19s 45ms/step - loss: 0.0097 - accuracy: 0.9972 - val_loss: 0.1272 - val_accuracy:

In [7]:
model.evaluate(x_train, y_train)

1875/1875 [==============================] - 9s 5ms/step - loss: 0.0270 - accuracy: 0.9952


[0.02704673632979393, 0.9951500296592712]

### 3. MaxPool
: featuere map을 구성할 때 필터에 해당하는 영역의 값 중 가장 큰 값만 가져와 
overfitting을 방지하고 사이즈를 반으로 줄여준다.

In [10]:
# 모델 생성
X = tf.keras.Input(shape=[28,28,1])
H = tf.keras.layers.Conv2D(filters =3 ,kernel_size=5,activation='swish') (X)
H = tf.keras.layers.MaxPool2D() (H)
H = tf.keras.layers.Conv2D(6 ,5,activation='swish') (H)
H = tf.keras.layers.MaxPool2D() (H)
H = tf.keras.layers.Flatten()(H)
H = tf.keras.layers.Dense(84,activation='swish') (H)
Y = tf.keras.layers.Dense(10,activation='softmax') (H)

model = tf.keras.Model(X,Y)
model.compile(loss="categorical_crossentropy", metrics="accuracy")
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 24, 24, 3)         78        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 3)        0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 8, 8, 6)           456       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 4, 4, 6)          0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 96)                0   

In [11]:
# 모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.1)

Epoch 1/10
422/422 [==============================] - 11s 23ms/step - loss: 2.4418 - accuracy: 0.8007 - val_loss: 0.2123 - val_accuracy: 0.9452
Epoch 2/10
422/422 [==============================] - 10s 25ms/step - loss: 0.1857 - accuracy: 0.9475 - val_loss: 0.1429 - val_accuracy: 0.9610
Epoch 3/10
422/422 [==============================] - 11s 27ms/step - loss: 0.1180 - accuracy: 0.9657 - val_loss: 0.1013 - val_accuracy: 0.9725
Epoch 4/10
422/422 [==============================] - 10s 24ms/step - loss: 0.0927 - accuracy: 0.9723 - val_loss: 0.0960 - val_accuracy: 0.9705
Epoch 5/10
422/422 [==============================] - 10s 24ms/step - loss: 0.0760 - accuracy: 0.9769 - val_loss: 0.0862 - val_accuracy: 0.9757
Epoch 6/10
422/422 [==============================] - 13s 30ms/step - loss: 0.0681 - accuracy: 0.9795 - val_loss: 0.0864 - val_accuracy: 0.9760
Epoch 7/10
422/422 [==============================] - 12s 27ms/step - loss: 0.0607 - accuracy: 0.9816 - val_loss: 0.0776 - val_accuracy:

In [12]:
# 모델 평가
model.evaluate(x_train, y_train)

1875/1875 [==============================] - 8s 4ms/step - loss: 0.0437 - accuracy: 0.9861


[0.043732695281505585, 0.986133337020874]